In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc

# PLV based  adjacency matrix loading

In [ ]:
#Load data from the drive location
X= np.load("E:/ADHD200/plv_corr/all_plv_adjacency.npy")
Y= np.load("E:/ADHD200/label_905.npy")

# GCN based classification on PLV based BFC with pytorch geometry

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
import time

# Convert the data to PyTorch tensors
x_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(Y, dtype=torch.long)

# Create the edge index for the fully connected graph
num_rois = X.shape[1]
row, col = [], []
for i in range(num_rois):
    for j in range(i + 1, num_rois):
        row.append(i)
        col.append(j)
edge_index = torch.tensor([row, col], dtype=torch.long)

# Create a custom GCN model
class GCNModel(nn.Module):
    def __init__(self, num_rois, num_psd_features, num_classes):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(num_psd_features, 128)
        self.conv2 = GCNConv(128, 64)
        self.conv3 = GCNConv(64, 32)
        self.fc = nn.Linear(num_rois * 32, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x

# Initialize the GCN model
num_psd_features = X.shape[2]  # Number of PSD features
num_classes = 2  # Binary classification
model = GCNModel(num_rois, num_psd_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders for batch training
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for all folds
precision_scores = []
recall_scores = []
specificity_scores = []
accuracy_scores = []
auc_scores = []
mean_fpr = np.linspace(0, 1, 100)
y_prob_all = []

# Mean ROC curve data
mean_tpr = 0.0

for fold, (train_val_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\n=================== Fold {fold + 1}/{num_folds} ===================\n")
    
    # Start timer for the entire fold
    fold_start_time = time.time()

    # Split the data into training+validation and testing sets for this fold
    x_train_val, x_test = x_tensor[train_val_idx], x_tensor[test_idx]
    y_train_val, y_test = y_tensor[train_val_idx], y_tensor[test_idx]
    
    # Further split the training data into 80% training and 20% validation
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_val, y_train_val, test_size=0.2, random_state=42
    )
    
    # Create the data and data loaders for this fold
    train_data = Data(x=x_train, edge_index=edge_index, y=y_train)
    val_data = Data(x=x_val, edge_index=edge_index, y=y_val)
    test_data = Data(x=x_test, edge_index=edge_index, y=y_test)
    
    train_loader = DataLoader([train_data], batch_size=16, shuffle=True)
    val_loader = DataLoader([val_data], batch_size=16, shuffle=False)
    test_loader = DataLoader([test_data], batch_size=16, shuffle=False)

    # Track time for training, validation, and testing separately
    train_time = 0
    val_time = 0
    test_time = 0

    # Train the model for this fold
    num_epochs = 100
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0.0
        epoch_start_time = time.time()  # Start time for this epoch
        for batch in train_loader:
            optimizer.zero_grad()
            output = model(batch.x, batch.edge_index)
            loss = criterion(output, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        train_time += time.time() - epoch_start_time  # Add training time for this epoch

        # Print the loss value for the current epoch
        print(f"Fold {fold + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}")

        # Validation phase
        model.eval()
        val_start_time = time.time()  # Start validation time
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in val_loader:
                output = model(batch.x, batch.edge_index)
                _, val_predicted_labels = torch.max(output, 1)
                val_correct += (val_predicted_labels == batch.y).sum().item()
                val_total += batch.y.size(0)
        val_time += time.time() - val_start_time  # Add validation time

        val_accuracy = val_correct / val_total
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy  # Save the best validation accuracy

    print(f"Fold {fold + 1} Best Validation Accuracy: {best_val_accuracy:.4f}")

    # Testing phase
    model.eval()
    test_start_time = time.time()  # Start testing time
    with torch.no_grad():
        output_test = model(x_test, edge_index)
        _, predicted_labels = torch.max(output_test, 1)
    test_time += time.time() - test_start_time  # Add testing time

    # Calculate the accuracy for this fold
    correct = (predicted_labels == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct / total
    print(f"Fold {fold + 1} Test Accuracy: {accuracy:.4f}")
    accuracy_scores.append(accuracy)

    # Calculate the AUC and ROC curve for this fold
    output_prob = torch.softmax(output_test, dim=1)[:, 1].numpy()
    y_prob_all.extend(output_prob)
    auc_score = roc_auc_score(y_test.numpy(), output_prob)
    auc_scores.append(auc_score)

    # Compute the evaluation metrics for this fold
    conf_matrix = confusion_matrix(y_test.numpy(), predicted_labels.numpy())
    tn, fp, fn, tp = conf_matrix.ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision_scores.append(precision)
    recall_scores.append(recall)
    specificity_scores.append(specificity)

    # Compute the ROC curve for this fold
    fpr, tpr, _ = roc_curve(y_test.numpy(), output_prob)
    mean_tpr += np.interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    plt.plot(fpr, tpr, alpha=0.3, label=f"Fold {fold + 1} (AUC = {auc_score:.2f})")

    # End timer for fold
    fold_end_time = time.time()
    print(f"Training Time for Fold {fold + 1}: {train_time:.2f} seconds")
    print(f"Validation Time for Fold {fold + 1}: {val_time:.2f} seconds")
    print(f"Testing Time for Fold {fold + 1}: {test_time:.2f} seconds")
    print(f"Total Time for Fold {fold + 1}: {fold_end_time - fold_start_time:.2f} seconds")

# Plot the mean ROC curve
mean_tpr /= num_folds
plt.plot(mean_fpr, mean_tpr, color='b', label=f"Mean ROC (AUC = {np.mean(auc_scores):.2f})", linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# Compute the mean and standard deviation of the evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

mean_precision = np.mean(precision_scores)
std_precision = np.std(precision_scores)

mean_recall = np.mean(recall_scores)
std_recall = np.std(recall_scores)

mean_specificity = np.mean(specificity_scores)
std_specificity = np.std(specificity_scores)

mean_auc = np.mean(auc_scores)

print(f"\nOverall Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
print(f"Overall Precision: {mean_precision:.4f} ± {std_precision:.4f}")
print(f"Overall Recall: {mean_recall:.4f} ± {std_recall:.4f}")
print(f"Overall Specificity: {mean_specificity:.4f} ± {std_specificity:.4f}")
print(f"Overall AUC: {mean_auc:.4f}")


# Increase feature size from 128 to 256 

In [ ]:
# Convert the data to PyTorch tensors
x_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(Y, dtype=torch.long)

# Create the edge index for the fully connected graph
num_rois = X.shape[1]
row, col = [], []
for i in range(num_rois):
    for j in range(i + 1, num_rois):
        row.append(i)
        col.append(j)
edge_index = torch.tensor([row, col], dtype=torch.long)

# Create a custom GCN model
class GCNModel(nn.Module):
    def __init__(self, num_rois, num_psd_features, num_classes):
        super(GCNModel, self).__init__()
        #total three GCN layers
        self.conv1 = GCNConv(num_psd_features, 256)
        self.conv2 = GCNConv(256, 64)
        self.conv3 = GCNConv(64, 32)
        self.fc = nn.Linear(num_rois * 32, num_classes)

    def forward(self, x, edge_index):
        # total three hidden layers
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x

# Initialize the GCN model
num_psd_features = X.shape[2]  # Number of PSD features
num_classes = 2  # Binary classification
model = GCNModel(num_rois, num_psd_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders for batch training
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for all folds
precision_scores = []
recall_scores = []
specificity_scores = []
accuracy_scores = []
auc_scores = []
mean_fpr = np.linspace(0, 1, 100)
y_prob_all = []

# Mean ROC curve data
mean_tpr = 0.0

for fold, (train_val_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\n=================== Fold {fold + 1}/{num_folds} ===================\n")
    
    # Start timer for the entire fold
    fold_start_time = time.time()

    # Split the data into training+validation and testing sets for this fold
    x_train_val, x_test = x_tensor[train_val_idx], x_tensor[test_idx]
    y_train_val, y_test = y_tensor[train_val_idx], y_tensor[test_idx]
    
    # Further split the training data into 80% training and 20% validation
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_val, y_train_val, test_size=0.2, random_state=42
    )
    
    # Create the data and data loaders for this fold
    train_data = Data(x=x_train, edge_index=edge_index, y=y_train)
    val_data = Data(x=x_val, edge_index=edge_index, y=y_val)
    test_data = Data(x=x_test, edge_index=edge_index, y=y_test)
    
    train_loader = DataLoader([train_data], batch_size=16, shuffle=True)
    val_loader = DataLoader([val_data], batch_size=16, shuffle=False)
    test_loader = DataLoader([test_data], batch_size=16, shuffle=False)

    # Track time for training, validation, and testing separately
    train_time = 0
    val_time = 0
    test_time = 0

    # Train the model for this fold
    num_epochs = 100
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0.0
        epoch_start_time = time.time()  # Start time for this epoch
        for batch in train_loader:
            optimizer.zero_grad()
            output = model(batch.x, batch.edge_index)
            loss = criterion(output, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        train_time += time.time() - epoch_start_time  # Add training time for this epoch

        # Print the loss value for the current epoch
        print(f"Fold {fold + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}")

        # Validation phase
        model.eval()
        val_start_time = time.time()  # Start validation time
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in val_loader:
                output = model(batch.x, batch.edge_index)
                _, val_predicted_labels = torch.max(output, 1)
                val_correct += (val_predicted_labels == batch.y).sum().item()
                val_total += batch.y.size(0)
        val_time += time.time() - val_start_time  # Add validation time

        val_accuracy = val_correct / val_total
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy  # Save the best validation accuracy

    print(f"Fold {fold + 1} Best Validation Accuracy: {best_val_accuracy:.4f}")

    # Testing phase
    model.eval()
    test_start_time = time.time()  # Start testing time
    with torch.no_grad():
        output_test = model(x_test, edge_index)
        _, predicted_labels = torch.max(output_test, 1)
    test_time += time.time() - test_start_time  # Add testing time

    # Calculate the accuracy for this fold
    correct = (predicted_labels == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct / total
    print(f"Fold {fold + 1} Test Accuracy: {accuracy:.4f}")
    accuracy_scores.append(accuracy)

    # Calculate the AUC and ROC curve for this fold
    output_prob = torch.softmax(output_test, dim=1)[:, 1].numpy()
    y_prob_all.extend(output_prob)
    auc_score = roc_auc_score(y_test.numpy(), output_prob)
    auc_scores.append(auc_score)

    # Compute the evaluation metrics for this fold
    conf_matrix = confusion_matrix(y_test.numpy(), predicted_labels.numpy())
    tn, fp, fn, tp = conf_matrix.ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision_scores.append(precision)
    recall_scores.append(recall)
    specificity_scores.append(specificity)

    # Compute the ROC curve for this fold
    fpr, tpr, _ = roc_curve(y_test.numpy(), output_prob)
    mean_tpr += np.interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    plt.plot(fpr, tpr, alpha=0.3, label=f"Fold {fold + 1} (AUC = {auc_score:.2f})")

    # End timer for fold
    fold_end_time = time.time()
    print(f"Training Time for Fold {fold + 1}: {train_time:.2f} seconds")
    print(f"Validation Time for Fold {fold + 1}: {val_time:.2f} seconds")
    print(f"Testing Time for Fold {fold + 1}: {test_time:.2f} seconds")
    print(f"Total Time for Fold {fold + 1}: {fold_end_time - fold_start_time:.2f} seconds")

# Plot the mean ROC curve
mean_tpr /= num_folds
plt.plot(mean_fpr, mean_tpr, color='b', label=f"Mean ROC (AUC = {np.mean(auc_scores):.2f})", linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# Compute the mean and standard deviation of the evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

mean_precision = np.mean(precision_scores)
std_precision = np.std(precision_scores)

mean_recall = np.mean(recall_scores)
std_recall = np.std(recall_scores)

mean_specificity = np.mean(specificity_scores)
std_specificity = np.std(specificity_scores)

mean_auc = np.mean(auc_scores)

print(f"\nOverall Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
print(f"Overall Precision: {mean_precision:.4f} ± {std_precision:.4f}")
print(f"Overall Recall: {mean_recall:.4f} ± {std_recall:.4f}")
print(f"Overall Specificity: {mean_specificity:.4f} ± {std_specificity:.4f}")
print(f"Overall AUC: {mean_auc:.4f}")



# Increase GCN layer 

In [ ]:
# Convert the data to PyTorch tensors
x_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(Y, dtype=torch.long)

# Create the edge index for the fully connected graph
num_rois = X.shape[1]
row, col = [], []
for i in range(num_rois):
    for j in range(i + 1, num_rois):
        row.append(i)
        col.append(j)
edge_index = torch.tensor([row, col], dtype=torch.long)

# Create a custom GCN model
class GCNModel(nn.Module):
    def __init__(self, num_rois, num_psd_features, num_classes):
        super(GCNModel, self).__init__()
        #total three GCN layers
        self.conv1 = GCNConv(num_psd_features, 256)
        self.conv2 = GCNConv(256, 64)
        self.conv3 = GCNConv(64, 64)
        self.conv3 = GCNConv(64, 32)
        self.fc = nn.Linear(num_rois * 32, num_classes)

    def forward(self, x, edge_index):
        # total three hidden layers
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x

# Initialize the GCN model
num_psd_features = X.shape[2]  # Number of PSD features
num_classes = 2  # Binary classification
model = GCNModel(num_rois, num_psd_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders for batch training
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for all folds
precision_scores = []
recall_scores = []
specificity_scores = []
accuracy_scores = []
auc_scores = []
mean_fpr = np.linspace(0, 1, 100)
y_prob_all = []

# Mean ROC curve data
mean_tpr = 0.0

for fold, (train_val_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\n=================== Fold {fold + 1}/{num_folds} ===================\n")
    
    # Start timer for the entire fold
    fold_start_time = time.time()

    # Split the data into training+validation and testing sets for this fold
    x_train_val, x_test = x_tensor[train_val_idx], x_tensor[test_idx]
    y_train_val, y_test = y_tensor[train_val_idx], y_tensor[test_idx]
    
    # Further split the training data into 80% training and 20% validation
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_val, y_train_val, test_size=0.2, random_state=42
    )
    
    # Create the data and data loaders for this fold
    train_data = Data(x=x_train, edge_index=edge_index, y=y_train)
    val_data = Data(x=x_val, edge_index=edge_index, y=y_val)
    test_data = Data(x=x_test, edge_index=edge_index, y=y_test)
    
    train_loader = DataLoader([train_data], batch_size=16, shuffle=True)
    val_loader = DataLoader([val_data], batch_size=16, shuffle=False)
    test_loader = DataLoader([test_data], batch_size=16, shuffle=False)

    # Track time for training, validation, and testing separately
    train_time = 0
    val_time = 0
    test_time = 0

    # Train the model for this fold
    num_epochs = 100
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0.0
        epoch_start_time = time.time()  # Start time for this epoch
        for batch in train_loader:
            optimizer.zero_grad()
            output = model(batch.x, batch.edge_index)
            loss = criterion(output, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        train_time += time.time() - epoch_start_time  # Add training time for this epoch

        # Print the loss value for the current epoch
        print(f"Fold {fold + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}")

        # Validation phase
        model.eval()
        val_start_time = time.time()  # Start validation time
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in val_loader:
                output = model(batch.x, batch.edge_index)
                _, val_predicted_labels = torch.max(output, 1)
                val_correct += (val_predicted_labels == batch.y).sum().item()
                val_total += batch.y.size(0)
        val_time += time.time() - val_start_time  # Add validation time

        val_accuracy = val_correct / val_total
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy  # Save the best validation accuracy

    print(f"Fold {fold + 1} Best Validation Accuracy: {best_val_accuracy:.4f}")

    # Testing phase
    model.eval()
    test_start_time = time.time()  # Start testing time
    with torch.no_grad():
        output_test = model(x_test, edge_index)
        _, predicted_labels = torch.max(output_test, 1)
    test_time += time.time() - test_start_time  # Add testing time

    # Calculate the accuracy for this fold
    correct = (predicted_labels == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct / total
    print(f"Fold {fold + 1} Test Accuracy: {accuracy:.4f}")
    accuracy_scores.append(accuracy)

    # Calculate the AUC and ROC curve for this fold
    output_prob = torch.softmax(output_test, dim=1)[:, 1].numpy()
    y_prob_all.extend(output_prob)
    auc_score = roc_auc_score(y_test.numpy(), output_prob)
    auc_scores.append(auc_score)

    # Compute the evaluation metrics for this fold
    conf_matrix = confusion_matrix(y_test.numpy(), predicted_labels.numpy())
    tn, fp, fn, tp = conf_matrix.ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision_scores.append(precision)
    recall_scores.append(recall)
    specificity_scores.append(specificity)

    # Compute the ROC curve for this fold
    fpr, tpr, _ = roc_curve(y_test.numpy(), output_prob)
    mean_tpr += np.interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    plt.plot(fpr, tpr, alpha=0.3, label=f"Fold {fold + 1} (AUC = {auc_score:.2f})")

    # End timer for fold
    fold_end_time = time.time()
    print(f"Training Time for Fold {fold + 1}: {train_time:.2f} seconds")
    print(f"Validation Time for Fold {fold + 1}: {val_time:.2f} seconds")
    print(f"Testing Time for Fold {fold + 1}: {test_time:.2f} seconds")
    print(f"Total Time for Fold {fold + 1}: {fold_end_time - fold_start_time:.2f} seconds")

# Plot the mean ROC curve
mean_tpr /= num_folds
plt.plot(mean_fpr, mean_tpr, color='b', label=f"Mean ROC (AUC = {np.mean(auc_scores):.2f})", linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# Compute the mean and standard deviation of the evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

mean_precision = np.mean(precision_scores)
std_precision = np.std(precision_scores)

mean_recall = np.mean(recall_scores)
std_recall = np.std(recall_scores)

mean_specificity = np.mean(specificity_scores)
std_specificity = np.std(specificity_scores)

mean_auc = np.mean(auc_scores)

print(f"\nOverall Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
print(f"Overall Precision: {mean_precision:.4f} ± {std_precision:.4f}")
print(f"Overall Recall: {mean_recall:.4f} ± {std_recall:.4f}")
print(f"Overall Specificity: {mean_specificity:.4f} ± {std_specificity:.4f}")
print(f"Overall AUC: {mean_auc:.4f}")


# Correlation based adjacency matrix loading

In [ ]:
#Load data from the drive location
X= np.load("E:/ADHD200/plv_corr/all_corr_adjacency.npy")
Y= np.load("E:/ADHD200/label_905.npy")


# GCN based classification on Correlation based BFC with pytorch geometry

In [ ]:
# Convert the data to PyTorch tensors
x_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(Y, dtype=torch.long)

# Create the edge index for the fully connected graph
num_rois = X.shape[1]
row, col = [], []
for i in range(num_rois):
    for j in range(i + 1, num_rois):
        row.append(i)
        col.append(j)
edge_index = torch.tensor([row, col], dtype=torch.long)

# Create a custom GCN model
class GCNModel(nn.Module):
    def __init__(self, num_rois, num_psd_features, num_classes):
        super(GCNModel, self).__init__()
        #total three GCN layers
        self.conv1 = GCNConv(num_psd_features, 128)
        self.conv2 = GCNConv(128, 64)
        self.conv3 = GCNConv(64, 32)
        self.fc = nn.Linear(num_rois * 32, num_classes)

    def forward(self, x, edge_index):
        # total three hidden layers
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x

# Initialize the GCN model
num_psd_features = X.shape[2]  # Number of PSD features
num_classes = 2  # Binary classification
model = GCNModel(num_rois, num_psd_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders for batch training
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for all folds
precision_scores = []
recall_scores = []
specificity_scores = []
accuracy_scores = []
auc_scores = []
mean_fpr = np.linspace(0, 1, 100)
y_prob_all = []

# Mean ROC curve data
mean_tpr = 0.0

for fold, (train_val_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\n=================== Fold {fold + 1}/{num_folds} ===================\n")
    
    # Start timer for the entire fold
    fold_start_time = time.time()

    # Split the data into training+validation and testing sets for this fold
    x_train_val, x_test = x_tensor[train_val_idx], x_tensor[test_idx]
    y_train_val, y_test = y_tensor[train_val_idx], y_tensor[test_idx]
    
    # Further split the training data into 80% training and 20% validation
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_val, y_train_val, test_size=0.2, random_state=42
    )
    
    # Create the data and data loaders for this fold
    train_data = Data(x=x_train, edge_index=edge_index, y=y_train)
    val_data = Data(x=x_val, edge_index=edge_index, y=y_val)
    test_data = Data(x=x_test, edge_index=edge_index, y=y_test)
    
    train_loader = DataLoader([train_data], batch_size=16, shuffle=True)
    val_loader = DataLoader([val_data], batch_size=16, shuffle=False)
    test_loader = DataLoader([test_data], batch_size=16, shuffle=False)

    # Track time for training, validation, and testing separately
    train_time = 0
    val_time = 0
    test_time = 0

    # Train the model for this fold
    num_epochs = 100
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0.0
        epoch_start_time = time.time()  # Start time for this epoch
        for batch in train_loader:
            optimizer.zero_grad()
            output = model(batch.x, batch.edge_index)
            loss = criterion(output, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        train_time += time.time() - epoch_start_time  # Add training time for this epoch

        # Print the loss value for the current epoch
        print(f"Fold {fold + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}")

        # Validation phase
        model.eval()
        val_start_time = time.time()  # Start validation time
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in val_loader:
                output = model(batch.x, batch.edge_index)
                _, val_predicted_labels = torch.max(output, 1)
                val_correct += (val_predicted_labels == batch.y).sum().item()
                val_total += batch.y.size(0)
        val_time += time.time() - val_start_time  # Add validation time

        val_accuracy = val_correct / val_total
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy  # Save the best validation accuracy

    print(f"Fold {fold + 1} Best Validation Accuracy: {best_val_accuracy:.4f}")

    # Testing phase
    model.eval()
    test_start_time = time.time()  # Start testing time
    with torch.no_grad():
        output_test = model(x_test, edge_index)
        _, predicted_labels = torch.max(output_test, 1)
    test_time += time.time() - test_start_time  # Add testing time

    # Calculate the accuracy for this fold
    correct = (predicted_labels == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct / total
    print(f"Fold {fold + 1} Test Accuracy: {accuracy:.4f}")
    accuracy_scores.append(accuracy)

    # Calculate the AUC and ROC curve for this fold
    output_prob = torch.softmax(output_test, dim=1)[:, 1].numpy()
    y_prob_all.extend(output_prob)
    auc_score = roc_auc_score(y_test.numpy(), output_prob)
    auc_scores.append(auc_score)

    # Compute the evaluation metrics for this fold
    conf_matrix = confusion_matrix(y_test.numpy(), predicted_labels.numpy())
    tn, fp, fn, tp = conf_matrix.ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision_scores.append(precision)
    recall_scores.append(recall)
    specificity_scores.append(specificity)

    # Compute the ROC curve for this fold
    fpr, tpr, _ = roc_curve(y_test.numpy(), output_prob)
    mean_tpr += np.interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    plt.plot(fpr, tpr, alpha=0.3, label=f"Fold {fold + 1} (AUC = {auc_score:.2f})")

    # End timer for fold
    fold_end_time = time.time()
    print(f"Training Time for Fold {fold + 1}: {train_time:.2f} seconds")
    print(f"Validation Time for Fold {fold + 1}: {val_time:.2f} seconds")
    print(f"Testing Time for Fold {fold + 1}: {test_time:.2f} seconds")
    print(f"Total Time for Fold {fold + 1}: {fold_end_time - fold_start_time:.2f} seconds")

# Plot the mean ROC curve
mean_tpr /= num_folds
plt.plot(mean_fpr, mean_tpr, color='b', label=f"Mean ROC (AUC = {np.mean(auc_scores):.2f})", linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# Compute the mean and standard deviation of the evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

mean_precision = np.mean(precision_scores)
std_precision = np.std(precision_scores)

mean_recall = np.mean(recall_scores)
std_recall = np.std(recall_scores)

mean_specificity = np.mean(specificity_scores)
std_specificity = np.std(specificity_scores)

mean_auc = np.mean(auc_scores)

print(f"\nOverall Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
print(f"Overall Precision: {mean_precision:.4f} ± {std_precision:.4f}")
print(f"Overall Recall: {mean_recall:.4f} ± {std_recall:.4f}")
print(f"Overall Specificity: {mean_specificity:.4f} ± {std_specificity:.4f}")
print(f"Overall AUC: {mean_auc:.4f}")

